In [1]:
import json
import gzip
from collections import defaultdict
from typing import Dict, List, Set
import random
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [2]:
# ---------------------------------------
# Utility Loader
# ---------------------------------------

def load_json(path: str):
    with gzip.open(path, "rt", encoding="utf-8") as f:
        return json.load(f)

In [3]:
class UserBasedCFRecommender():
    """
    User-based kNN Collaborative Filtering using SPARSE cosine similarity.
    Trained only on train_user_book interactions.
    """

    def __init__(self, train_user_book: Dict[str, List[str]], k_neighbors: int = 50):
        # Deduplicate interactions
        self.train_user_book = {u: list(set(books)) for u, books in train_user_book.items()}
        self.k_neighbors = k_neighbors

        self.user_ids = []
        self.book_ids = []
        self.user_id_map = {}
        self.book_id_map = {}
        self.interaction_matrix = None
        self.user_sim_matrix = None

    def fit(self):
        print("Building User-Based CF (Sparse, train-only)...")

        user_book = self.train_user_book

        # Collect ids
        self.user_ids = sorted(user_book.keys())
        self.book_ids = sorted({b for books in user_book.values() for b in books})

        self.user_id_map = {u: i for i, u in enumerate(self.user_ids)}
        self.book_id_map = {b: i for i, b in enumerate(self.book_ids)}

        # Build sparse interaction matrix
        rows, cols = [], []
        print("Building sparse interaction matrix...")
        for u, books in tqdm(user_book.items()):
            ui = self.user_id_map[u]
            for b in books:
                bi = self.book_id_map[b]
                rows.append(ui)
                cols.append(bi)

        data = np.ones(len(rows), dtype=np.float32)
        self.interaction_matrix = csr_matrix(
            (data, (rows, cols)),
            shape=(len(self.user_ids), len(self.book_ids)),
        )

        # User-user cosine similarity
        print("Computing user-user similarity (this may take 1–3 minutes)...")
        self.user_sim_matrix = cosine_similarity(
            self.interaction_matrix, dense_output=False
        )

    def recommend(self, user_id: str, k: int = 10):
        if user_id not in self.user_id_map:
            return []

        uidx = self.user_id_map[user_id]
        user_sim = self.user_sim_matrix.getrow(uidx).toarray().ravel()

        # k nearest neighbours (excluding self)
        neighbor_idxs = np.argsort(user_sim)[::-1][1 : self.k_neighbors + 1]

        scores = np.zeros(len(self.book_ids), dtype=np.float32)
        for nidx in neighbor_idxs:
            scores += self.interaction_matrix.getrow(nidx).toarray().ravel()

        # Filter out items already seen in TRAIN
        seen = self.interaction_matrix.getrow(uidx).toarray().ravel() > 0
        scores[seen] = -1

        top_items = np.argsort(scores)[::-1][:k]
        return [self.book_ids[i] for i in top_items]

In [4]:
class RankingEvaluator:
    """
    Evaluates ranking-based recommendation metrics.
    Computes: Hit@K, MRR@K, NDCG@K for various K values.
    """
    
    def __init__(self):
        pass
    
    def hit_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        """
        Hit@K: 1 if at least one relevant item is in top-K, else 0
        """
        top_k = predictions[:k]
        return 1.0 if any(item in ground_truth for item in top_k) else 0.0
    
    def mrr_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        """
        MRR@K: Reciprocal rank of first relevant item in top-K
        """
        top_k = predictions[:k]
        for rank, item in enumerate(top_k, start=1):
            if item in ground_truth:
                return 1.0 / rank
        return 0.0
    
    def dcg_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        """
        DCG@K: Discounted Cumulative Gain
        """
        dcg = 0.0
        top_k = predictions[:k]
        for rank, item in enumerate(top_k, start=1):
            if item in ground_truth:
                dcg += 1.0 / np.log2(rank + 1)
        return dcg
    
    def idcg_at_k(self, ground_truth: Set[str], k: int) -> float:
        """
        IDCG@K: Ideal DCG (best possible DCG)
        """
        ideal_k = min(len(ground_truth), k)
        idcg = sum(1.0 / np.log2(rank + 1) for rank in range(1, ideal_k + 1))
        return idcg
    
    def ndcg_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        """
        NDCG@K: Normalized Discounted Cumulative Gain
        """
        dcg = self.dcg_at_k(predictions, ground_truth, k)
        idcg = self.idcg_at_k(ground_truth, k)
        
        if idcg == 0.0:
            return 0.0
        
        return dcg / idcg
    
    def evaluate(self, predictions: Dict[str, List[str]], ground_truth: Dict[str, Set[str]]) -> Dict[str, float]:
        """
        Evaluate predictions against ground truth.
        
        Args:
            predictions: Dict mapping user_id -> list of recommended item_ids (ranked)
            ground_truth: Dict mapping user_id -> set of relevant item_ids
            
        Returns:
            Dict of metric_name -> average_score
        """
        metrics = {
            'Hit@5': [],
            'Hit@10': [],
            'Hit@50': [],
            'MRR@10': [],
            'NDCG@10': [],
            'NDCG@50': []
        }
        
        # Only evaluate users present in both predictions and ground_truth
        common_users = set(predictions.keys()) & set(ground_truth.keys())
        
        for user_id in common_users:
            preds = predictions[user_id]
            gt = ground_truth[user_id]
            
            # Skip users with no ground truth items
            if len(gt) == 0:
                continue
            
            # Compute metrics
            metrics['Hit@5'].append(self.hit_at_k(preds, gt, 5))
            metrics['Hit@10'].append(self.hit_at_k(preds, gt, 10))
            metrics['Hit@50'].append(self.hit_at_k(preds, gt, 50))
            metrics['MRR@10'].append(self.mrr_at_k(preds, gt, 10))
            metrics['NDCG@10'].append(self.ndcg_at_k(preds, gt, 10))
            metrics['NDCG@50'].append(self.ndcg_at_k(preds, gt, 50))
        
        # Average across all users
        results = {}
        for metric_name, values in metrics.items():
            if len(values) > 0:
                results[metric_name] = np.mean(values)
            else:
                results[metric_name] = 0.0
        
        return results

In [5]:
def build_user_book_interactions(
    user_to_review_path: str,
    book_to_review_path: str,
) -> Dict[str, List[str]]:
    """
    Builds:
        user_id -> [book_id, book_id, ...]
    """
    user_to_review = load_json(user_to_review_path)
    book_to_review = load_json(book_to_review_path)

    # Normalize IDs to string
    review_to_user = {
        str(x["review_id"]): str(x["user_id"])
        for x in user_to_review
    }

    review_to_book = {
        str(x["review_id"]): str(x["book_id"])
        for x in book_to_review
    }

    user_book = defaultdict(list)

    for rid, user_id in review_to_user.items():
        if rid in review_to_book:
            book_id = review_to_book[rid]
            user_book[user_id].append(book_id)

    return user_book


def split_user_interactions(
    user_book: Dict[str, List[str]],
    seed: int = 42,
):
    """
    Returns:
        train_user_book
        val_user_book
        test_user_book
    """
    random.seed(seed)

    train = {}
    val = {}
    test = {}

    for user, books in user_book.items():
        books = list(set(books))  # remove duplicates
        random.shuffle(books)

        n = len(books)
        if n < 3:
            train[user] = books
            val[user] = []
            test[user] = []
            continue

        n_train = int(0.7 * n)
        n_val = int(0.15 * n)

        train[user] = books[:n_train]
        val[user] = books[n_train : n_train + n_val]
        test[user] = books[n_train + n_val :]

    return train, val, test


def build_ground_truth(test_user_book: Dict[str, List[str]]) -> Dict[str, Set[str]]:
    return {
        user: set(books)
        for user, books in test_user_book.items()
        if len(books) > 0
    }


def generate_predictions(model, users: List[str], k: int):
    predictions = {}

    for u in tqdm(users, desc="Generating Predictions"):
        preds = model.recommend(u, k)
        predictions[u] = preds

    return predictions

In [6]:
def main():
    # -------------------------------
    # Paths
    # -------------------------------

    DATA_FOLDER = "RokomariBG_Dataset/"
    USER_TO_REVIEW = DATA_FOLDER+"user_to_review.json.gz"
    BOOK_TO_REVIEW = DATA_FOLDER+"book_to_review.json.gz"

    K = 10

    # -------------------------------
    # Build user-book interactions
    # -------------------------------
    user_book = build_user_book_interactions(
        USER_TO_REVIEW,
        BOOK_TO_REVIEW,
    )

    print(f"Total users with interactions: {len(user_book)}")

    evaluator = RankingEvaluator()

    # -------------------------------
    # Split 70/15/15
    # -------------------------------
    train_user_book, val_user_book, test_user_book = split_user_interactions(
        user_book
    )

    ground_truth = build_ground_truth(test_user_book)
    test_users = list(ground_truth.keys())

    print(f"Users in test set: {len(test_users)}")

    # =====================================================
    # USER-BASED CF
    # =====================================================

    print("\nTraining User-Based CF...")

    user_cf = UserBasedCFRecommender(
        train_user_book=train_user_book,
        k_neighbors=50,
    )
    user_cf.fit()

    user_cf_preds = generate_predictions(user_cf, test_users, K)
    user_cf_metrics = evaluator.evaluate(user_cf_preds, ground_truth)

    print("\n===== User-Based CF Results =====")
    for m, v in user_cf_metrics.items():
        print(f"{m}: {v:.4f}")


if __name__ == "__main__":
    main()

Total users with interactions: 63721
Users in test set: 15427

Training User-Based CF...
Building User-Based CF (Sparse, train-only)...
Building sparse interaction matrix...


100%|███████████████████████████████████████████████████████████████████████| 63721/63721 [00:00<00:00, 1012698.25it/s]

Computing user-user similarity (this may take 1–3 minutes)...



Generating Predictions: 100%|███████████████████████████████████████████████████| 15427/15427 [01:14<00:00, 205.80it/s]



===== User-Based CF Results =====
Hit@5: 0.1009
Hit@10: 0.1235
Hit@50: 0.1235
MRR@10: 0.0749
NDCG@10: 0.0659
NDCG@50: 0.0657
